In [13]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
from dash.exceptions import PreventUpdate
import pandas as pd
from textblob import TextBlob
import io
import base64
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import dash_table

import matplotlib.pyplot as plt
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [14]:
# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LUX])

In [15]:
app.layout = html.Div([
    html.H3("Sentiment Analysis Dashboard", style={'textAlign': 'left'}),
    dcc.Upload(
        id='upload-data',
        children=html.Button('Upload File'),
        multiple=False
    ),
    html.Div(id='pie-chart-container'),
    html.Div(id='dropdown-container'),  # Create a container for the dropdown
    html.Div(id='output-data-upload'),
    html.Div(id='sentence-display'),
])#, style={'display': 'flex', 'flexDirection': 'column', 'alignItems': 'center', 'justifyContent': 'flex-start'})

def generate_table(df, column_name, score_column_name):
    df = df.sort_values(by=score_column_name, ascending=False)
    table = dash_table.DataTable(
        data=df[[column_name, score_column_name]].to_dict('records'),
        columns=[{'name': column_name, 'id': column_name},
                  {'name': score_column_name, 'id': score_column_name}],
        style_table={'overflowX': 'auto', 'maxWidth': '60%'},
        style_cell={'maxWidth': '100px', 'whiteSpace': 'normal'}
    )
    return table

In [16]:
# Define a callback to display/hide the dropdown based on file uploads
@app.callback(
    Output('dropdown-container', 'children'),
    Input('upload-data', 'contents')
)
def toggle_dropdown_visibility(uploaded_files):
    if uploaded_files:
        return dcc.Dropdown(
            id='select-table',
            options=[
                {'label': 'Q8 Which part(s) of the course was MOST valuable to you? Please explain why.', 'value': 'Q8'},
                {'label': 'Q9 Which part(s) of the course was LEAST valuable to you? Please explain why.', 'value': 'Q9'},
                {'label': 'Q10 Please provide any other comments or suggestions you have for improving this course.', 'value': 'Q10'}
            ],
            value='None',
            style={'width': '750px'}
        )  # Show the dropdown when files are uploaded
    else:
        return html.Div()

In [17]:
# Define a callback to update the sentence display based on the selected label
@app.callback(
    Output('sentence-display', 'children'),
    Input('select-table', 'value')
)
def update_sentence_display(selected_label):
    sentences = {
        'Q8': 'Note: Value range：0 to 1（from less positive to more positive) and ranking from the most positive feedback',
        'Q9': 'Note: Value range：0 to 1（from less negative to more negative）and ranking from the most negative feedback',
        'Q10': 'Note: Value range：-1 to 1（from negative to positive）and ranking from the most positive feedback'
    }
    return sentences.get(selected_label, '')

In [ ]:
@app.callback(
    Output('pie-chart-container', 'children'),
    Input('select-table', 'value')
)

def generate_pie_chart(df):
    # Categorize the scores into three groups
    def categorize_score(score):
        if score < 0:
            return "Negative"
        elif score == 0:
            return "Neutral"
        else:
            return "Positive"
    
    df['Score_Category'] = df['Q8_Positive Sentiment Score'].apply(categorize_score)
    df['Score_Category'] = df['Q9_Negative Sentiment Score''].apply(categorize_score)
    df['Score_Category'] = df['Q10_Compound Sentiment Score'].apply(categorize_score)

    # Count the number of occurrences for each category
    category_counts = df['Score_Category'].value_counts()

    # Create a pie chart
    plt.figure(figsize=(8, 8))
    plt.pie(category_counts, labels=category_counts.index, autopct='%1.1f%%', startangle=140)
    plt.title('Distribution of Sentiment Scores')
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle

In [19]:
# Define a callback to connect dropdown menu with the cleaned dataset
@app.callback(
    Output('output-data-upload', 'children'),
    Output('pie-chart-container', 'children'),  # Add this line to update the pie chart
    [Input('upload-data', 'contents'),
     Input('select-table', 'value')]
)
def update_output(contents, selected_table):
    if contents is None:
        return "Upload a file"

    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string).decode('utf-8')

    # Assuming your file is in CSV format
    df = pd.read_csv(io.StringIO(decoded))

    # Data cleaning
    if 'Q8' in df.columns and 'Q9' in df.columns and 'Q10' in df.columns:
        df = df[['Q8', 'Q9', 'Q10']]  # Keep only these three columns

    df = df.iloc[2:]  # Remove the first two rows

    # Drop rows with missing values in 'Q8', 'Q9', and 'Q10' columns
    df.dropna(subset=['Q8', 'Q9', 'Q10'], inplace=True)

    # Initialize SentimentIntensityAnalyzer
    analyzer = SentimentIntensityAnalyzer()

    # Convert 'Q8' column to strings to avoid TypeError
    df['Q8'] = df['Q8'].astype(str)

    # Convert 'Q9' column to strings to avoid TypeError
    df['Q9'] = df['Q9'].astype(str)

    # Convert 'Q10' column to strings to avoid TypeError
    df['Q10'] = df['Q10'].astype(str)

    # Calculate sentiment scores for each column
    df['Q8_Positive Sentiment Score'] = df['Q8'].apply(lambda x: analyzer.polarity_scores(x)['pos'])
    df['Q9_Negative Sentiment Score'] = df['Q9'].apply(lambda x: analyzer.polarity_scores(x)['neg'])
    df['Q10_Compound Sentiment Score'] = df['Q10'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

    # Sort the DataFrame by sentiment score columns in descending order
    df = df.sort_values(by=['Q8_Positive Sentiment Score'], ascending=False)

    # Create three separate tables with equal width
    table_q8 = dash_table.DataTable(
        data=df[['Q8', 'Q8_Positive Sentiment Score']].to_dict('records'),
        columns=[{'name': 'Q8', 'id': 'Q8'}, {'name': 'Positive Sentiment Score', 'id': 'Q8_Positive Sentiment Score'}],
        style_table={'overflowX': 'auto', 'maxWidth': '70%'},  # Adjust the 'maxWidth' as needed
    style_cell={'maxWidth': '100px', 'whiteSpace': 'normal'}  # Adjust the 'maxWidth' and 'whiteSpace' as needed
    )
    
    # Sort the DataFrame by sentiment score columns in descending order
    df = df.sort_values(by='Q9_Negative Sentiment Score', ascending=False)
    
    table_q9 = dash_table.DataTable(
        data=df[['Q9', 'Q9_Negative Sentiment Score']].to_dict('records'),
        columns=[{'name': 'Q9', 'id': 'Q9'}, {'name': 'Negative Sentiment Score', 'id': 'Q9_Negative Sentiment Score'}],
        style_table={'overflowX': 'auto', 'maxWidth': '70%'},  # Adjust the 'maxWidth' as needed
    style_cell={'maxWidth': '100px', 'whiteSpace': 'normal'}  # Adjust the 'maxWidth' and 'whiteSpace' as needed
    )
    
    # Sort the DataFrame by sentiment score columns in descending order
    df = df.sort_values(by='Q10_Compound Sentiment Score', ascending=False)

    table_q10 = dash_table.DataTable(
        data=df[['Q10', 'Q10_Compound Sentiment Score']].to_dict('records'),
        columns=[{'name': 'Q10', 'id': 'Q10'}, {'name': 'Compound Sentiment Score', 'id': 'Q10_Compound Sentiment Score'}],
        style_table={'overflowX': 'auto', 'maxWidth': '70%'},  # Adjust the 'maxWidth' as needed
    style_cell={'maxWidth': '100px', 'whiteSpace': 'normal'}  # Adjust the 'maxWidth' and 'whiteSpace' as needed
    )

    if selected_table == 'Q8':
        return generate_table(df, 'Q8', 'Q8_Positive Sentiment Score')
    elif selected_table == 'Q9':
        return generate_table(df, 'Q9', 'Q9_Negative Sentiment Score')
    elif selected_table == 'Q10':
        generate_pie_chart(df)  # Generate the pie chart
        return generate_table(df, 'Q10', 'Q10_Compound Sentiment Score')

In [20]:
if __name__ == '__main__':
    app.run_server(port=2010)
    print(f'Running on http://127.0.0.1:2010/')

Running on http://127.0.0.1:2010/


[2023-10-14 21:01:55,056] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/yyaxuan/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/yyaxuan/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/yyaxuan/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/yyaxuan/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/Users/yyaxuan/opt/anaconda3/lib/python3.9/site-packages/dash/dash.py", line 1310, in dispatch
    ctx.run(
  File "/Users/yyaxuan/opt/anaconda3/lib/python3.9/site-packages/dash/_callback.py", line 457, in add_context
    flat_output_